## 0. Import Packages

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## 1. Load MNIST (csv)

In [2]:
train_data = pd.read_csv("mnist_train.csv")
test_data = pd.read_csv("mnist_test.csv")

In [4]:
print("Train Shape :", train_data.shape)
print("Test Shape ", test_data.shape)

Train Shape : (60000, 785)
Test Shape  (10000, 785)


In [5]:
X_train = train_data.drop(["label"], axis = 1).values
X_test = test_data.drop(["label"], axis = 1).values

y_train = train_data.label.values
y_test = test_data.label.values

In [6]:
print("X Train Shape :", X_train.shape)
print("X Test Shape ", X_test.shape)
print("y Train Shape :",y_train.shape)
print("y Test Shape ",y_test.shape)

X Train Shape : (60000, 784)
X Test Shape  (10000, 784)
y Train Shape : (60000,)
y Test Shape  (10000,)


## 2. RBM (784 -> 196 (14*14) (75% reduction))

In [47]:
# hyper parameters
n_visible = 28*28
n_hidden = 196
display_step = 10
num_epochs = 100
batch_size = 256
lr = tf.constant(0.001, tf.float32)

In [48]:
x = tf.placeholder(tf.float32, [None, n_visible], name = 'x')
w = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name = 'w')
b_h = tf.Variable(tf.zeros([1, n_hidden], tf.float32, name = 'bh'))
b_v = tf.Variable(tf.zeros([1, n_visible], tf.float32, name = 'bv'))

In [49]:
def next_batch(num, data, labels):
    
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [50]:
def sample(prob):
    return tf.floor(prob + tf.random_uniform(tf.shape(prob), 0, 1))

def gibbs_step(x_k):
    h_k = sample(tf.sigmoid(tf.matmul(x_k, w) + b_h))
    x_k = sample(tf.sigmoid(tf.matmul(h_k, tf.transpose(w)) + b_v))
    return x_k

def gibbs_sample(k, x_k):
    for i in range(k):
        x_out = gibbs_step(x_k)
    return x_out

x_s = gibbs_sample(2, x)
h_s = sample(tf.sigmoid(tf.matmul(x_s, w) + b_h))

h = sample(tf.sigmoid(tf.matmul(x, w) + b_h))
x_ = sample(tf.sigmoid(tf.matmul(h, tf.transpose(w)) + b_v))

size_batch = tf.cast(tf.shape(x)[0], tf.float32)
w_add = tf.multiply(lr/size_batch, tf.subtract(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_s), h_s)))
bv_add = tf.multiply(lr/size_batch, tf.reduce_sum(tf.subtract(x, x_s), 0, True))
bh_add = tf.multiply(lr/size_batch, tf.reduce_sum(tf.subtract(h, h_s), 0, True))
updt = [w.assign_add(w_add), b_v.assign_add(bv_add), b_h.assign_add(bh_add)]

In [51]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    total_batch = int(X_train.shape[0]/batch_size)
    for epoch in range(num_epochs):
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(batch_size, X_train, y_train)
            batch_xs = (batch_xs>0)*1
            _ = sess.run([updt], feed_dict = {x:batch_xs})
        if epoch % display_step == 0:
            print("Epoch :", epoch+1)
    print("done")
    out1 = sess.run(h, feed_dict = {x: X_train})
    out2 = sess.run(h, feed_dict = {x: X_test})

Epoch : 1
Epoch : 11
Epoch : 21
Epoch : 31
Epoch : 41
Epoch : 51
Epoch : 61
Epoch : 71
Epoch : 81
Epoch : 91
done


In [52]:
print("train shape :", out1.shape)
print("test shape :", out2.shape)

train shape : (60000, 196)
test shape : (10000, 196)


In [53]:
df1 = pd.DataFrame(out1)
df1.to_csv('x_train_rbm.csv', index=False)
df2 = pd.DataFrame(out2)
df2.to_csv('x_test_rbm.csv', index=False)

## 3. Data load test

In [54]:
rbm_train = pd.read_csv('x_train_rbm.csv')
rbm_test = pd.read_csv('x_test_rbm.csv')

In [55]:
print("train shape :", rbm_train.shape)
print("test shape :", rbm_test.shape)

train shape : (60000, 196)
test shape : (10000, 196)
